# Word embeddings
* 단어를 벡터로 표현하는 것을 말한다.
* One hot vector == 희소 벡터(sparse representation)로 단어를 표현할 수 있다.
* Dense Representation(밀집 표현) : 벡터 차원을 단어들의 갯수(클래스의 수)로 하지 않고 실수를 이용해 벡터를 표현하는 방법
* 밀집 벡터(dense vector)의 형태로 표현하는 방법을 **워드 임베딩(Word Embedding)** 이라고 한다.

In [8]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [2]:
embedding_layer = layers.Embedding(1000, 5)  

`tf.keras.layers.Embedding(
    input_dim, output_dim, embeddings_initializer='uniform',
    embeddings_regularizer=None, activity_regularizer=None,
    embeddings_constraint=None, mask_zero=False, input_length=None, **kwargs
)`

**input_dim** : 단어 집합의 크기, 즉 총 단어의 개수  
**output_dim** : 임베딩 벡터의 출력 차원. 결과로 나오는 임베딩 벡터의 크기  
**input_length** : 입력 시퀀스의 길이(입력되는 단어 벡터의 dim) `지정하지 않으면 입력으로 지정`  

**Input shape** : 2D tensor with shape: `(batch_size, input_length)`  
**Output shape** : 3D tensor with shape: `(batch_size, input_length, output_dim)`  

In [9]:
result = embedding_layer(tf.constant([1,2,3]))

In [18]:
result

<tf.Tensor: id=15, shape=(3, 5), dtype=float32, numpy=
array([[ 0.03873748,  0.04885865, -0.01484647,  0.02384646, -0.00934587],
       [ 0.00692027,  0.02759692,  0.0427526 , -0.01029741,  0.01278073],
       [ 0.00243457, -0.01415724, -0.03165891,  0.03213947, -0.04175957]],
      dtype=float32)>

# 전처리(Preprocessing)
**Tokenizer() :** 토큰화와 정수 인코딩(단어에 대한 인덱싱)을 위해 keras에서 제공하는 Tokenizer를 사용한다.

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
t  = Tokenizer()
fit_text = "The earth is an awesome place live"
t.fit_on_texts([fit_text])

test_text = "The earth is an great place live"
sequences = t.texts_to_sequences([test_text])[0]

print("sequences : ",sequences) # great는 단어 집합(vocabulary)에 없으므로 출력되지 않는다.
print("word_index : ",t.word_index) # 단어 집합(vocabulary) 출력

sequences :  [1, 2, 3, 4, 6, 7]
word_index :  {'the': 1, 'earth': 2, 'is': 3, 'an': 4, 'awesome': 5, 'place': 6, 'live': 7}


**pad_sequence()** : 전체 훈련 모델의 샘플 길이는 다르기 때문에 다른 데이터와의 통일성을 위해 padding을 넣어준다.

In [23]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
pad_sequences([[1, 2, 3], [3, 4, 5, 6], [7, 8]], maxlen=3, padding='pre')
# 전처리가 끝나서 각 단어에 대한 정수 인코딩이 끝났다고 가정하고, 3개의 데이터를 입력으로 합니다.

array([[1, 2, 3],
       [4, 5, 6],
       [0, 7, 8]])

**첫번째 인자 = 패딩을 진행할 데이터**

**maxlen = 모든 데이터에 대해서 정규화할 길이**

**padding = 'pre'는 앞에 pad를 채우고 'post'는 뒤에 pad를 채움**

**Embedding()** : Embedding()는 단어를 밀집 벡터로 만드는 역할을 한다. 입력은 (number of samples, input_length)인 2D를 입력받고 출력은 3D 인 (number of samples, input_length, embedding word dimentionality)를 반환한다.

In [29]:
# 문장 토큰화와 단어 토큰화
text=[['Hope', 'to', 'see', 'you', 'soon'],['Nice', 'to', 'see', 'you', 'again']]

# 각 단어에 대한 정수 인코딩
text=[[0, 1, 2, 3, 4],[5, 1, 2, 3, 6]]

# 위 데이터가 아래의 임베딩 층의 입력이 된다.
result = layers.Embedding(7, 2, input_length=5)
# 7은 단어의 개수. 즉, 단어 집합(vocabulary)의 크기이다.
# 2는 임베딩한 후의 벡터의 크기이다.
# 5는 각 입력 시퀀스의 길이. 즉, input_length이다.
'''
# 각 정수는 아래의 테이블의 인덱스로 사용되며 Embeddig()은 각 단어에 대해 임베딩 벡터를 리턴한다.
+------------+------------+
|   index    | embedding  |
+------------+------------+
|     0      | [1.2, 3.1] |
|     1      | [0.1, 4.2] |
|     2      | [1.0, 3.1] |
|     3      | [0.3, 2.1] |
|     4      | [2.2, 1.4] |
|     5      | [0.7, 1.7] |
|     6      | [4.1, 2.0] |
+------------+------------+
# 위의 표는 임베딩 벡터가 된 결과를 예로서 정리한 것이고 Embedding()의 출력인 3D 텐서를 보여주는 것이 아님.
'''

'\n# 각 정수는 아래의 테이블의 인덱스로 사용되며 Embeddig()은 각 단어에 대해 임베딩 벡터를 리턴한다.\n+------------+------------+\n|   index    | embedding  |\n+------------+------------+\n|     0      | [1.2, 3.1] |\n|     1      | [0.1, 4.2] |\n|     2      | [1.0, 3.1] |\n|     3      | [0.3, 2.1] |\n|     4      | [2.2, 1.4] |\n|     5      | [0.7, 1.7] |\n|     6      | [4.1, 2.0] |\n+------------+------------+\n# 위의 표는 임베딩 벡터가 된 결과를 예로서 정리한 것이고 Embedding()의 출력인 3D 텐서를 보여주는 것이 아님.\n'

AttributeError: 'Embedding' object has no attribute 'numpy'